In [6]:
import numpy as np
from matplotlib import gridspec
import matplotlib.pyplot as plt
%matplotlib inline
from astropy import units as u
from astropy.coordinates import SkyCoord 
from astropy.io import ascii
from astropy.units import Quantity
from astroquery.gaia import Gaia
import os
import warnings
warnings.filterwarnings("ignore")

# Setup for Search and Save

In [18]:
snr_list[1][0]

'Tycho'

In [7]:
#os.chdir('C:\\Users\\robin\\Documents\\Research')
os.chdir('D:\\Documents\Research\Supernova Remnant Data')
t = 1
snr_list = ascii.read('snrsdata.txt')
snr_name = snr_list[t][0] # G0.0+0.0 Name
snr_ra = snr_list[t][1]   # 00:00:00 RA
snr_dec = snr_list[t][2]  # 00:00:00 Dec
snr_size = snr_list[t][3] # radius in arc-minutes
snr_ra_dec = SkyCoord(snr_ra, snr_dec, unit = (u.deg, u.deg), frame = 'icrs')
snr_size = snr_size * u.arcmin
snr_size_deg = snr_size.to(u.deg)
os.chdir('D:\\Documents\Research\Supernova Remnant Data/' + snr_name)
print('Loaded SNR: ' + snr_name)

Loaded SNR: Tycho


# Searching for Stars

## Single Search and Save

In [8]:
#ra,dec,size = (snr_ra_dec.ra.deg[t], snr_ra_dec.dec.deg[t], (snr_size_deg[t].value)/2.0)
size_offset = 10.0
ra,dec,size = (snr_ra_dec.ra.deg, snr_ra_dec.dec.deg, size_offset*(snr_size_deg.value))
job = Gaia.launch_job_async("SELECT TOP 10000000 * \
FROM gaiadr2.gaia_source \
WHERE (CONTAINS(POINT('ICRS',ra,dec),CIRCLE('ICRS', %s, %s, %s))=1 AND parallax_over_error >= 3 \
AND phot_g_mean_flux_over_error >= 5 AND phot_rp_mean_flux_over_error >=5 AND phot_bp_mean_flux_over_error >= 5)" % (ra,dec,size) , dump_to_file=True)
print (snr_name)
print (job)
r = job.get_results()
ra_and_dec = SkyCoord(ra = r['ra'], dec = r['dec'], unit= (u.deg,u.deg) )
dist_from_center = ra_and_dec.separation(snr_ra_dec).arcmin
inside_remnant = np.where(dist_from_center <= snr_size.value)[0]
outside_remnant = np.where(dist_from_center > snr_size.value)[0]
ascii.write(r[inside_remnant], str(snr_name)+' potential stars.txt', format = 'csv' )
ascii.write(r[outside_remnant], str(snr_name)+' sample stars.txt', format = 'csv' )
print (len(r))
print (len(r[inside_remnant]))
print (len(r[outside_remnant]))

Tycho
Jobid: 1571184293183O
Phase: COMPLETED
Owner: None
Output file: async_20191015170453.vot
Results: None
18691
173
18518


# Grabbing Samples

In [28]:
s1 = "SELECT TOP 400000 parallax, parallax_error, phot_g_mean_mag,phot_g_mean_flux, phot_g_mean_flux_error, bp_rp,phot_rp_mean_flux, phot_rp_mean_flux_error, phot_bp_mean_flux,phot_bp_mean_flux_error, e_bp_min_rp_val, a_g_val \
FROM gaiadr2.gaia_source \
WHERE (1/parallax >= 2 AND 1/parallax <= 8 AND parallax_over_error >= 3 AND phot_g_mean_flux_over_error >= 5 AND phot_rp_mean_flux_over_error >=5 AND phot_bp_mean_flux_over_error >= 5)"

In [12]:
ra,dec,size = (snr_ra_dec.ra.deg[0]+2, snr_ra_dec.dec.deg[0]+4, (3))
s1 = "SELECT TOP 400000 parallax, parallax_error, pmra, pmra_error, pmdec, pmdec_error \
FROM gaiadr2.gaia_source \
WHERE (CONTAINS(POINT('ICRS',ra,dec),CIRCLE('ICRS', %s, %s, %s))=1 AND 1/parallax >= 2 AND 1/parallax <= 8 AND parallax_over_error >= 3 AND phot_g_mean_flux_over_error >= 5 AND phot_rp_mean_flux_over_error >=5 AND phot_bp_mean_flux_over_error >= 5)"

In [13]:
job = Gaia.launch_job_async(s1 % (ra,dec,size), dump_to_file=True)
print (job)
r = job.get_results()
ascii.write(r, 'velocity_sample_data.txt', format = 'csv' )
print (len(r))

Jobid: 1552002177553O
Phase: COMPLETED
Owner: None
Output file: async_20190307154259.vot
Results: None
400000


In [30]:
distance = 1/r['parallax']*1000

In [31]:
near_snr = np.where((distance >= 2000) & (distance <=8000))
print len(near_snr[0])

400000


# All Search

In [1]:
#for i in range(len(snr_name)):
#    print snr_name[i]
#    ra,dec,size = (snr_ra_dec.ra.deg[i], snr_ra_dec.dec.deg[i], (snr_size_deg[i].value)/2.0)
#    job = Gaia.launch_job_async("SELECT TOP 10000000 * \
#FROM gaiadr2.gaia_source \
#WHERE CONTAINS(POINT('ICRS',ra,dec),CIRCLE('ICRS', %s, %s, %s))=1" % (ra,dec,size) ,
#                      dump_to_file=True)
#    print job
#    r = job.get_results()
#    ascii.write(r, str(snr_name[i])+ '_' + str((snr_size[i].value/2.0)) + '_arcmin.txt', format = 'csv' )
#    print len(r)

In [3]:
len(snr_name)

253

In [18]:
print snr_name[1]
print snr_ra_dec[1]
print snr_size[1]

G27.8+0.6
<SkyCoord (ICRS): (ra, dec) in deg
    (279.95833333, -4.4)>
50.0 arcmin


In [ ]:
(snr_size_deg[0].value)/2.0

In [22]:
print snr_ra[t]
print snr_dec[t]

15:02:50
-41:56:00


In [26]:
snr_name[9]

'G4.5+6.8'

In [12]:
test = ascii.read('G299.2-2.9.txt')
print len(test[0][0:])

41646


# Remnants that crashed my comp

G13.3-1.3 ~ 560,000 stars in 35 arcmin
G25.1-2.3 ~ 1,000,000 stars in 40 arcmin